## Q2. Use  the  Admission dataset to perform the following task. 

### PART-2

#### Compare  the  performances  of  logistic  regression  model  withKNN model on the Admission dataset

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
dataset = pd.read_csv("../input_data/AdmissionDataset/data.csv")
df_r, df_c = dataset.shape #without drop
print "Dataset: ", df_r, df_c
target = 'Chance of Admit '

Dataset:  450 9


In [3]:
dataset = dataset.drop(['Serial No.'], axis=1)

In [4]:
#normalise dataframe
def normalise(df):
    cols = df.columns
    for c in cols:
        if c != target:
            sd = df[c].std()
            mean = df[c].mean()
            df[c] = (df[c] - mean)/sd
    return df
dataset = normalise(dataset)
# print dataset.head()

   GRE Score  TOEFL Score  University Rating       SOP      LOR       CGPA  \
0   0.040384    -0.690991          -0.988084 -0.866855 -1.597605 -0.713316   
1   0.216817     0.139084          -0.111086 -0.363520  0.033837 -0.062724   
2   0.481468     0.471114          -0.111086  0.139815 -1.053791  0.154140   
3   0.834335     0.969158           1.642909  1.146486  0.577651  1.371916   
4   0.216817    -0.192946          -0.111086  0.139815 -1.053791 -0.413043   

   Research  Chance of Admit   
0 -1.111779              0.65  
1  0.897460              0.71  
2  0.897460              0.80  
3  0.897460              0.91  
4  0.897460              0.74  


Finds classes from chance of admit for ENTIRE dataset

In [5]:
def findClasses(row, threshold, default=0):
    if row[target] <= threshold:
        label = 0
    else:
        label = 1
    return label

In [6]:
def helper_classes(df, predicted_class):
    threshold = 0.5
    df[predicted_class] = df.apply(findClasses, axis=1, args=(threshold,0,))
    return df[predicted_class]
dataset['class'] = helper_classes(dataset, 'class')
# print dataset.head()

   GRE Score  TOEFL Score  University Rating       SOP      LOR       CGPA  \
0   0.040384    -0.690991          -0.988084 -0.866855 -1.597605 -0.713316   
1   0.216817     0.139084          -0.111086 -0.363520  0.033837 -0.062724   
2   0.481468     0.471114          -0.111086  0.139815 -1.053791  0.154140   
3   0.834335     0.969158           1.642909  1.146486  0.577651  1.371916   
4   0.216817    -0.192946          -0.111086  0.139815 -1.053791 -0.413043   

   Research  Chance of Admit   class  
0 -1.111779              0.65      1  
1  0.897460              0.71      1  
2  0.897460              0.80      1  
3  0.897460              0.91      1  
4  0.897460              0.74      1  


In [7]:
# dataset = dataset.sample(frac=1)
train, validate = np.split(dataset, [int(.8*len(dataset))])
train_rows, train_cols = train.shape
print train_rows, train_cols
# print validate.head()

360 9
     GRE Score  TOEFL Score  University Rating       SOP      LOR       CGPA  \
360  -0.930001    -1.189036          -0.988084 -0.363520 -0.509977 -0.963544   
361  -1.900387    -1.853096          -0.988084 -1.873525 -1.597605 -2.064546   
362  -0.841785    -0.358961          -0.988084 -1.370190  0.033837 -0.796725   
363  -0.224267    -0.358961          -0.988084 -0.866855 -1.597605 -1.564091   
364   0.216817     0.803143          -0.111086 -0.866855 -1.597605  0.220868   

     Research  Chance of Admit   class  
360 -1.111779              0.48      0  
361 -1.111779              0.47      0  
362 -1.111779              0.53      1  
363 -1.111779              0.70      1  
364  0.897460              0.78      1  


In [8]:
def findEuclidean(unknown_class, known_class):

    cols = ['GRE Score','TOEFL Score','University Rating','SOP','LOR ','CGPA','Research']
    sum_of_squares = 0
    class_dist = []
    for i in cols:
        if i != target:
            xi = (unknown_class[i] - known_class[i])**2
            sum_of_squares += xi
    
    dist = math.sqrt(sum_of_squares)
    class_dist = [dist, known_class['class']]

    return class_dist


In [9]:
def findKNeighbours(class_dist, k):
    
    topK = class_dist[0:k]

    count0 = 0
    count1 = 0   
    #traverse the second list and count no of 1,0
    for i in topK:
        if i[1] == 0:
            count0 += 1
        elif i[1] == 1:
            count1 += 1
    if count0 > count1:
        label = 0
    else:
        label = 1
#     print "label: ",label
    return label

In [10]:
def findKnn(row, df_t, findNeigh, distanceMeasure, k, default=0):
    listOfList=[]

    #iterate on each row of training set.
    for i, r in df_t.iterrows():
        
        temp = distanceMeasure(row,r) #temp: [ <distance>, <class>]
        listOfList.append(temp)    
    listOfList.sort()

    pred = findNeigh(listOfList, k)

    return pred


def helper_knn(df, df_t, predict_col, findNeigh, distanceMeasure, k):
    df[predict_col] = df.apply(findKnn, axis=1, args=(df_t, findNeigh, distanceMeasure, k, 0))
    return df[predict_col]
validate['predict_col'] = helper_knn(validate, train, 'predict_col', findKNeighbours, findEuclidean, 3)
# print validate.head()

     GRE Score  TOEFL Score  University Rating       SOP      LOR       CGPA  \
360  -0.930001    -1.189036          -0.988084 -0.363520 -0.509977 -0.963544   
361  -1.900387    -1.853096          -0.988084 -1.873525 -1.597605 -2.064546   
362  -0.841785    -0.358961          -0.988084 -1.370190  0.033837 -0.796725   
363  -0.224267    -0.358961          -0.988084 -0.866855 -1.597605 -1.564091   
364   0.216817     0.803143          -0.111086 -0.866855 -1.597605  0.220868   

     Research  Chance of Admit   class  predict_col  
360 -1.111779              0.48      0            1  
361 -1.111779              0.47      0            0  
362 -1.111779              0.53      1            1  
363 -1.111779              0.70      1            1  
364  0.897460              0.78      1            1  


In [11]:
#returns Confusion Matrix
def createCM(predicted, actual):

    pred = pd.Series(predicted, name='Predicted')

    actu = pd.Series(actual,    name='Actual')

    conf = pd.crosstab(actu, pred)

    return conf

In [12]:
#function to find accuracy, precision, recall
#parameter: confusion matrix
def findMeasures(mat):    
    diag = 0
    tot = 0
    for i in mat:
        diag += mat[i][i]
        tot += mat[i].sum()
    accuracy = float(diag)/tot

    precision = np.diag(mat) / np.sum(mat, axis = 0)
    recall = np.diag(mat) / np.sum(mat, axis = 1)
    f1_score_den = 1/precision + 1/recall
    f1_score = float(2)/f1_score_den
    return accuracy, precision, recall,f1_score

In [13]:
def completeAnalysis(df_v, df_t, predict_col, funNeigh, funDist, bestK):
    df_v[predict_col] = helper_knn(df_v, df_t, predict_col, funNeigh, funDist, bestK)
    
    predicted_val = df_v[predict_col]
    print "predicted: ",len(predicted_val)
    
    actual_val = df_v[target]
    print "actual val: ", len(actual_val)
    mean_actual = actual_val.mean()
    

    
    class_actual = df_v['class']
    confusion_mat = createCM(predicted_val, class_actual)
    print "Confusion Matrix:"
    print confusion_mat
    a,p,r,f = findMeasures(confusion_mat)
    print "Accuracy: ",a*100, "\nPrecision: ", p*100,"\nRecall:", r*100,"\nF1Score: ",f
    

In [14]:
completeAnalysis(validate, train, 'validate', findKNeighbours, findEuclidean, 3)

predicted:  90
actual val:  90
Confusion Matrix:
Predicted  0   1
Actual          
0          4   5
1          0  81
Accuracy:  94.44444444444444 
Precision:  Predicted
0    100.000000
1     94.186047
dtype: float64 
Recall: Actual
0     44.444444
1    100.000000
dtype: float64 
F1Score:  Predicted
0    0.615385
1    0.970060
dtype: float64
